In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

In [2]:
companies = ["liander", "stedin"]
features = ["num_connections", "perc_of_active_connections", "type_conn_perc", "annual_consume"]

## Type_connection_perc predict

In [3]:
train = pd.DataFrame()
for year in range(2011, 2020):
    data = pd.read_csv("gas/enexis_gas_%d.csv" % year)[features]
    train = train.append(data, ignore_index = True)
test = pd.read_csv("gas/enexis_gas_2010.csv")[features]

In [4]:
model = LinearRegression(n_jobs = 8)
scaler = MinMaxScaler()
train_X = scaler.fit_transform(train.iloc[:, [0,1,3]].values)
train_y = train.iloc[:, [2]].values

In [5]:
model.fit(train_X, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=8, normalize=False)

In [6]:
test["type_conn_perc"] = model.predict(scaler.transform(test.iloc[:, [0,1,3]].values))

In [7]:
test["type_conn_perc"] = test["type_conn_perc"].apply(lambda x: 100 if(x > 100) else (0 if(x < 0) else x))

In [8]:
data_2010 = pd.read_csv("gas/enexis_gas_2010.csv")
data_2010["type_conn_perc"] = test["type_conn_perc"]
data_2010.to_csv("gas/enexis_gas_2010.csv", index = False, float_format = "%g")

## Smartmeter predict

In [10]:
features = ["num_connections", "perc_of_active_connections", "type_conn_perc", "annual_consume", "smartmeter_perc"]

In [12]:
model = LinearRegression(n_jobs = 8)
scaler = MinMaxScaler()
for year in range(2010, 2020):
    train = pd.DataFrame()
    for company in companies:
        data = pd.read_csv("gas/%s_gas_%s.csv" % (company, year))[features]
        train = train.append(data, ignore_index = True)
    train = train.dropna().reset_index(drop = True)
    train_X = scaler.fit_transform(train.iloc[:, [0,1,2,3]].values)
    train_y = train.iloc[:,4].values
    model.fit(train_X, train_y)
    data = pd.read_csv("gas/enexis_gas_%s.csv" % year)
    test = pd.DataFrame(data[features])
    test_X = scaler.transform(test.iloc[:, [0,1,2,3]].values)
    test.iloc[:, 4] = model.predict(test_X)
    test.iloc[:, 4] = test.iloc[:, 4].apply(lambda x: 100 if(x > 100) else (0 if(x < 0) else x))
    data["smartmeter_perc"] = test.iloc[:, 4]
    data.to_csv("gas/enexis_gas_%s.csv" % year, index = False, float_format = "%g")